# Flowers Resnet Finetuning TPU

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import os
os.chdir("/content/drive/My Drive/ResnetTPU")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import tensorflow as tf
import json
# Auth on Colab (little wrinkle: without auth, Colab will be extremely slow in accessing data from a GCS bucket, even public).
IS_COLAB_BACKEND = 'COLAB_GPU' in os.environ  # this is always set on Colab, the value is 0 or 1 depending on GPU presence
if IS_COLAB_BACKEND:
  from google.colab import auth
  auth.authenticate_user()  # Starts an Auth prompt
  if 'COLAB_TPU_ADDR' in os.environ:  # if using a TPU backend
    with tf.Session('grpc://{}'.format(os.environ['COLAB_TPU_ADDR'])) as sess:    
      with open('rishabh-229507-693d810284ca.json', 'r') as auth_info:
        # Propagate the Colab Auth so that the TPU can access your GCS buckets. This is not necessary for
        # the public bucket used in this sample but it will be once you work with your own non-public data.
        tf.contrib.cloud.configure_gcs(sess, credentials=json.load(auth_info))  # Upload the credentials to TPU


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [3]:
!pip install cloud-tpu-profiler

    100% |████████████████████████████████| 4.2MB 3.7MB/s 


## Data preparation

Download and unzip the flowers dataset.

In [0]:
!wget http://download.tensorflow.org/example_images/flower_photos.tgz

In [0]:
!tar -xzf flower_photos.tgz

Split some data for testing and validation.

In [0]:
!pip install split-folders

In [0]:
import split_folders
split_folders.ratio("flower_photos", output="data")

In [0]:
#  %set_env GOOGLE_APPLICATION_CREDENTIALS=/content/drive/My Drive/InceptionTPU/rishabh-229507-693d810284ca.json

In [0]:
# !gcloud auth activate-service-account --key-file rishabh-229507-693d810284ca.json

In [0]:
!find flower_photos/dandelion -type f | wc -l
!find flower_photos/sunflowers -type f | wc -l
!find flower_photos/tulips -type f | wc -l
!find flower_photos/daisy -type f | wc -l
!find flower_photos/roses -type f | wc -l

898
699
799
633
641


In [0]:
!find data/train -type f | wc -l
!find data/validation -type f | wc -l
!find data/test -type f | wc -l

2934
364
372


In [0]:
ls data

test/  train/  validation/


In [0]:
!du -ah --max-depth=2 tf-records | sort -h

In [0]:
!python2 tpu/tools/datasets/imagenet_to_gcs.py \
  --project="Rishabh" \
  --gcs_output_path="gs://resnet-tpu-colab/data" \
  --raw_data_dir="data" \
  --local_scratch_dir="tf-records"

## Transfer learning

In [0]:
!gsutil ls gs://resnet-tpu-colab/

gs://resnet-tpu-colab/data/
gs://resnet-tpu-colab/resnet-gpu/


In [0]:
!git clone https://github.com/tensorflow/tpu.git

Cloning into 'tpu'...
remote: Enumerating objects: 80, done.
remote: Counting objects: 100% (80/80), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 2752 (delta 35), reused 56 (delta 26), pack-reused 2672
Receiving objects: 100% (2752/2752), 3.11 MiB | 6.04 MiB/s, done.
Resolving deltas: 100% (1677/1677), done.
Checking out files: 100% (263/263), done.


In [4]:
cd tpu/models/official/resnet

/content/drive/My Drive/ResnetTPU/tpu/models/official/resnet


In [5]:
%set_env PYTHONPATH=/env/python:/content/drive/My Drive/ResnetTPU/tpu/models

env: PYTHONPATH=/env/python:/content/drive/My Drive/ResnetTPU/tpu/models


In [6]:
!echo $PYTHONPATH

/env/python:/content/drive/My Drive/ResnetTPU/tpu/models


In [0]:
cat imagenet_input.py

In [0]:
# # test to run with unoptimized input pipeline
# TPU = 'grpc://' + os.environ['COLAB_TPU_ADDR']
# !python2 resnet_main.py \
#   --tpu=$TPU \
#   --data_dir="gs://resnet-tpu-colab/data" \
#   --param_overrides=train_steps=42,iterations_per_loop=6,num_parallel_calls=64,train_batch_size=1024 \
#   --model_dir="gs://resnet-tpu-colab/resnet-tpu-basic-model" \
#   --resnet_checkpoint="gs://cloud-tpu-artifacts/resnet/resnet-nhwc-2018-02-07" \
#   --mode=train \
#   --steps_per_eval=42 \
#   --profile_every_n_steps=1 \
#   --tpu_profile_duration=60000
# #   --export_dir="gs://resnet-tpu-colab/exported-model"

In [8]:
TPU = 'grpc://' + os.environ['COLAB_TPU_ADDR']
!python2 resnet_main.py \
  --tpu=$TPU \
  --data_dir="gs://resnet-tpu-colab/data" \
  --param_overrides=train_steps=42,iterations_per_loop=6,num_parallel_calls=64,train_batch_size=1024 \
  --model_dir="gs://resnet-tpu-colab/resnet-tpu-model" \
  --resnet_checkpoint="gs://cloud-tpu-artifacts/resnet/resnet-nhwc-2018-02-07" \
  --mode=train_and_eval \
  --steps_per_eval=42 \
  --profile_every_n_steps=1 \
  --tpu_profile_duration=15000 \
  --export_dir="gs://resnet-tpu-colab/exported-model"

I0325 19:22:27.353693 139972175525760 resnet_main.py:553] Model params: {'dropblock_keep_prob': 0.9, 'transpose_input': True, 'use_async_checkpointing': False, 'dropblock_groups': '', 'poly_rate': 0.0, 'dropblock_size': 7, 'precision': 'bfloat16', 'base_learning_rate': 0.1, 'use_cache': True, 'image_size': 299, 'train_batch_size': 1024, 'weight_decay': 0.0001, 'num_train_images': 2934, 'train_steps': 42, 'use_tpu': True, 'skip_host_call': False, 'eval_batch_size': 360, 'num_cores': 8, 'data_format': 'channels_last', 'resnet_depth': 50, 'num_label_classes': 5, 'num_parallel_calls': 64, 'iterations_per_loop': 6, 'num_eval_images': 364, 'label_smoothing': 0.0, 'momentum': 0.9, 'enable_lars': False}
I0325 19:22:27.355051 139972175525760 estimator.py:201] Using config: {'_save_checkpoints_secs': None, '_session_config': graph_options {
  rewrite_options {
    disable_meta_optimizer: true
  }
}
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.93.191.58:8470"

In [0]:
!python2 resnet_main.py \
  --tpu=$TPU \
  --data_dir="gs://resnet-tpu-colab/data" \
  --model_dir="gs://resnet-tpu-colab/resnet-tpu-model" \
  --mode=eval

In [9]:
!python2 resnet_main.py \
  --tpu=$TPU \
  --data_dir="gs://resnet-tpu-colab/data" \
  --model_dir="gs://resnet-tpu-colab/resnet-tpu-model" \
  --mode=predict \
  --predict_batch_size=360 \
  --num_test_images=372

I0325 19:26:54.431488 139948228130688 resnet_main.py:553] Model params: {'dropblock_keep_prob': 0.9, 'transpose_input': True, 'use_async_checkpointing': False, 'dropblock_groups': '', 'poly_rate': 0.0, 'dropblock_size': 7, 'precision': 'bfloat16', 'base_learning_rate': 0.1, 'use_cache': True, 'image_size': 299, 'train_batch_size': 1024, 'weight_decay': 0.0001, 'num_train_images': 2934, 'train_steps': 120, 'use_tpu': True, 'skip_host_call': False, 'eval_batch_size': 360, 'num_cores': 8, 'data_format': 'channels_last', 'resnet_depth': 50, 'num_label_classes': 5, 'num_parallel_calls': 8, 'iterations_per_loop': 3, 'num_eval_images': 364, 'label_smoothing': 0.0, 'momentum': 0.9, 'enable_lars': False}
I0325 19:26:54.432765 139948228130688 estimator.py:201] Using config: {'_save_checkpoints_secs': None, '_session_config': graph_options {
  rewrite_options {
    disable_meta_optimizer: true
  }
}
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.93.191.58:8470"

In [0]:
!unzip ngrok-stable-linux-amd64.zip

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ngrok                   


In [0]:
LOG_DIR = 'gs://resnet-tpu-colab/resnet-tpu/'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

get_ipython().system_raw('./ngrok http 6006 &')

!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://622a22e3.ngrok.io


In [0]:
####misc commands
# import subprocess
# subprocess.Popen(["capture_tpu_profile","--logdir=gs://resnet-tpu-colab/resnet-tpu-pipeline", "--service_addr={}".format(os.environ['COLAB_TPU_ADDR'])])
# !capture_tpu_profile --tpu="grpc://10.102.83.170:8470"  --monitoring_level=2